In [ ]:
# Install general packages
!pip install PyPDF2 streamlit python-dotenv

# Install LangChain and specific components (Spacy, FAISS, Anthropic, OpenAI)
!pip install langchain langchain-embeddings faiss-cpu anthropic openai

# Install additional components for Spacy
!pip install spacy

# Download Spacy language models
!python -m spacy download en_core_web_sm



ERROR: Could not find a version that satisfies the requirement langchain-embeddings (from versions: none)
ERROR: No matching distribution found for langchain-embeddings
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 83.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install streamlit PyPDF2 spacy faiss-cpu openai langchain python-dotenv
!python -m spacy download en_core_web_sm
!pip install langchain
!pip install -U langchain-community

In [ ]:
!ngrok authtoken 2mZj2Vm8W6vElwry9SuCKfaGQqu_3v9Cj8f7ps56SjfUVVnkm

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
sk-kfMJvRKJ2fDtE2kS2Ej0BstBgAmK1vqD4q7tlGANbnT3BlbkFJOt8bI2bRgioC9Q8mLhLXBhrkKJcJ5SC2IctVVGS5wA

In [18]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.48.0
    Uninstalling openai-1.48.0:
      Successfully uninstalled openai-1.48.0


In [25]:
!pip install --upgrade openai


  Using cached openai-1.48.0-py3-none-any.whl.metadata (24 kB)
Using cached openai-1.48.0-py3-none-any.whl (376 kB)
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0


In [36]:
# Step 1: Install the necessary libraries
!pip install streamlit PyPDF2 openai pdfplumber pyngrok

# Step 2: Write your Streamlit app to a Python file
with open('aiteacher.py', 'w') as f:
    f.write("""
import os
import streamlit as st
import pdfplumber
from pyngrok import ngrok
import subprocess
from openai import OpenAI  # Import the OpenAI client

# Set up OpenAI API key
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY', 'sk-kfMJvRKJ2fDtE2kS2Ej0BstBgAmK1vqD4q7tlGANbnT3BlbkFJOt8bI2bRgioC9Q8mLhLXBhrkKJcJ5SC2IctVVGS5wA'))

# Global variables to store PDF text, questions, and the current question index
pdf_text = ""
questions = []
current_question_index = 0
paragraphs = []

# Function to read PDF and extract text
def pdf_read(pdf_doc):
    text = ''
    with pdfplumber.open(pdf_doc) as pdf:
        for page in pdf.pages:
            text += page.extract_text()
    return text

# Function to clean and preprocess the extracted PDF text
def preprocess_text(text):
    text = text.replace('\\n', ' ')  # Remove unnecessary line breaks
    text = ' '.join(text.split())   # Remove extra spaces
    return text

# Function to generate relevant questions using GPT-3.5 Turbo (Chat model)
def generate_questions_gpt(paragraph):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that generates relevant questions based on a given paragraph."},
            {"role": "user", "content": f"Generate two contextually relevant and informative questions from the following paragraph: {paragraph}"}
        ],
        temperature=0.7,
        n=1,
        max_tokens=150
    )
    # Correctly extract the response content
    return response.choices[0].message.content.strip().split('\\n')

# Function to generate the correct answer using GPT-3.5 Turbo (Chat model)
def get_correct_answer(question, context):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that answers questions based on the given context."},
            {"role": "user", "content": f"Using the context from the following text, answer this question:\\n\\nText: {context}\\n\\nQuestion: {question}"}
        ],
        temperature=0.7,
        n=1,
        max_tokens=150
    )
    return response.choices[0].message.content.strip()

# Function to rate the user's answer using GPT-3.5 Turbo (Chat model)
def rate_answer(user_answer, correct_answer):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an evaluator that gives feedback on the user's answer."},
            {"role": "user", "content": f"Evaluate this user's answer based on its relevance and correctness:\\n\\nUser's Answer: {user_answer}\\nCorrect Answer: {correct_answer}\\nRate from 0 to 10 and provide an explanation."}
        ],
        temperature=0.7,
        n=1,
        max_tokens=150
    )
    return response.choices[0].message.content.strip()

# Streamlit app logic
def main():
    global pdf_text, questions, current_question_index, paragraphs

    st.set_page_config('AI Teacher: Q&A with PDF')
    st.header('Q&A with PDF')

    # Upload PDF file
    pdf_doc = st.file_uploader('Upload your PDF File, Chapter by Chapter', type=['pdf'])

    if pdf_doc:
        with st.spinner('Processing...'):
            pdf_text = pdf_read(pdf_doc)
            pdf_text = preprocess_text(pdf_text)  # Clean the text
            paragraphs = pdf_text.split('. ')  # Split the text into paragraphs
            questions = []

            # Generate questions from the first few paragraphs (for demonstration)
            for paragraph in paragraphs[:3]:  # Limit to the first 3 paragraphs for speed
                questions += generate_questions_gpt(paragraph)

            current_question_index = 0  # Reset index after new upload
            st.success('PDF processed and questions generated successfully!')

    if questions:
        # Display current question
        st.subheader('Question:')
        st.write(questions[current_question_index])

        # User input for the answer
        user_answer = st.text_input('Your Answer:')

        if st.button('Submit Answer'):
            # Get correct answer and rate the user's answer
            correct_answer = get_correct_answer(questions[current_question_index], pdf_text)
            rating_response = rate_answer(user_answer, correct_answer)

            st.write('Correct Answer:', correct_answer)
            st.write('Your Rating:', rating_response)

        if st.button('Next Question'):
            current_question_index = (current_question_index + 1) % len(questions)
            # Display the next question
            st.write(questions[current_question_index])

if __name__ == '__main__':
    main()

    """)

# Step 3: Run the Streamlit app with ngrok
from pyngrok import ngrok
import subprocess

# Start the Streamlit app
process = subprocess.Popen(['streamlit', 'run', 'aiteacher.py'])

# Open ngrok tunnel to the Streamlit port (8501)
public_url = ngrok.connect(8501)
print("Streamlit URL:", public_url)


Streamlit URL: NgrokTunnel: "https://a459-34-106-101-220.ngrok-free.app" -> "http://localhost:8501"
